In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

C:\Users\42128\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading the data

In [2]:
mnist_data, mnist_info = tfds.load('mnist', as_supervised=True, with_info=True)

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

### Separate the data

In [4]:
mnist_train_data, mnist_test_data = mnist_data['train'], mnist_data['test']

# get the number of train, validation, and test data
num_train_samples = mnist_info.splits['train'].num_examples
# take 10% from train data
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [5]:
print(num_validation_samples)

tf.Tensor(6000, shape=(), dtype=int64)


### Scaling 

In [6]:
# define a function to scale the data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [7]:
scaled_train_data = mnist_train_data.map(scale)
scaled_test_data = mnist_test_data.map(scale)

### Shuffling

In [8]:
# set the shuffle size
shuffle_size = 10000

# shuffled
shuffled_train_data = scaled_train_data.shuffle(shuffle_size)

# take 10% data from the train_data to vaidation data
validation_data = shuffled_train_data.take(num_validation_samples)
train_data = shuffled_train_data.skip(num_validation_samples)

### Batching

In [9]:
batch_size = 100

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

In [10]:
validation_input, validation_tagert = next(iter(validation_data))

## Modeling and Optimization

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 100

# create model include inputs, hidden layers, and output
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [12]:
customized_optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
iteration = 50
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data, validation_data=(validation_input, validation_tagert), callbacks=[early_stopping], 
          epochs=iteration, verbose=2)

Epoch 1/50
540/540 - 3s - loss: 0.3376 - accuracy: 0.9050 - val_loss: 0.1806 - val_accuracy: 0.9505 - 3s/epoch - 6ms/step
Epoch 2/50
540/540 - 2s - loss: 0.1358 - accuracy: 0.9601 - val_loss: 0.1197 - val_accuracy: 0.9655 - 2s/epoch - 5ms/step
Epoch 3/50
540/540 - 2s - loss: 0.0952 - accuracy: 0.9713 - val_loss: 0.1029 - val_accuracy: 0.9715 - 2s/epoch - 4ms/step
Epoch 4/50
540/540 - 3s - loss: 0.0744 - accuracy: 0.9772 - val_loss: 0.0710 - val_accuracy: 0.9802 - 3s/epoch - 5ms/step
Epoch 5/50
540/540 - 2s - loss: 0.0591 - accuracy: 0.9823 - val_loss: 0.0569 - val_accuracy: 0.9833 - 2s/epoch - 4ms/step
Epoch 6/50
540/540 - 2s - loss: 0.0478 - accuracy: 0.9851 - val_loss: 0.0548 - val_accuracy: 0.9812 - 2s/epoch - 4ms/step
Epoch 7/50
540/540 - 2s - loss: 0.0403 - accuracy: 0.9878 - val_loss: 0.0464 - val_accuracy: 0.9863 - 2s/epoch - 4ms/step
Epoch 8/50
540/540 - 2s - loss: 0.0336 - accuracy: 0.9899 - val_loss: 0.0435 - val_accuracy: 0.9870 - 2s/epoch - 4ms/step
Epoch 9/50
540/540 - 3s 

## Test the model

In [14]:
test_loss, test_accuracy = model.evaluate(test_data) 

1/1 [==============================] - 0s 243ms/step - loss: 0.0924 - accuracy: 0.9791


In [15]:
(test_accuracy*100)

97.9099988937378